## Imports:

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import keras
tfd = tfp.distributions

import libdnnmcmc.se_NN_lib as NN
from libdnnmcmc.utility import import_training_data, load_scenario, LogProbDemand
from libdnnmcmc.importance_sampling import weight_samples
from libdnnmcmc.evaluation_functions import ed, segmented_ed, beauty_report_qdists


In [2]:
''' ++++++++++++++++++ settings ++++++++++++++++++++++++++++++++++++++'''
testcase = 'loop'   # either 'loop' or 'tree'

# MCMC settings:
n_runs_MCMC = 2             # repeat calculations "n_runs_MCMC" times and average results
n_burn_ins = [2.e4]         # number of burn-in-steps during MCMC interferrence
num_chains = 10             # number of independent Markov-Chains, run in parallel
num_results = int(1.e4)     # number of samples drawn per chain

# evaluation settings: (calculating the energy distance is very demanding in terms of computational power and RAM)
calculate_ED = False

## Load Setup:

In [3]:
SE, VM, d_prior_dist, data_file = load_scenario(testcase)

# load trained DNN model
path = f'./models/model_{testcase}'
model = keras.models.load_model(path, custom_objects={'MyScalingLayer': NN.MyScalingLayer,
                                                      'MetricMAPE_T': NN.MetricMAPE_T,
                                                      'MetricMAPE_mf': NN.MetricMAPE_mf,
                                                      'MetricMAPE_p': NN.MetricMAPE_p,
                                                      'LossWeightedMSE': NN.LossWeightedMSE,
                                                      'MetricMAPE_Tend': NN.MetricMAPE_Tend})

# output settings:
file_spec = f'testcase_{testcase}'         # identifier appended to all output files

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


setup evaluation metrics & load MC results as ground truth:

In [4]:
n_nodes = SE.n_nodes
n_edges = SE.n_edges
n_demands = SE.n_demands
n_states = 2*n_nodes+2*n_edges

ED = np.zeros((n_runs_MCMC, 1))                     # Energy Distance
elementwise_ED = np.zeros((n_runs_MCMC, 4))         # Energy Distance indiv. for T, mf, p, Tend
q10dist = np.zeros((n_runs_MCMC, n_states))         # quantile error for 10% quantiles
q5dist = np.zeros((n_runs_MCMC, n_states))          # quantile error for  5% quantiles
mean_dist = np.zeros((n_runs_MCMC, n_states))       # mean value error
q5dist_rel = np.zeros((n_runs_MCMC, n_states))      # relative prediction distance for  5% quantiles
q10dist_rel = np.zeros((n_runs_MCMC, n_states))     # relative prediction distance for 10% quantiles

measurement_indices = VM.measurement_indices
measurement_stds = VM.measurement_noise
measurement_names = [(key + ' at ' + pair[0]) for key in VM.installed_measure.keys() for pair in VM.installed_measure[key]]
# the first 62.5k samples were used to train the DNN; exclude them for the result evaluation
# _, MC_states = import_demand_files(data_file, 262_500)
# MC_states = MC_states[62_500:]

_, MC_states = import_training_data(data_file, 200_000, skip=62_500)

## Perform MCMC interference for random Measurement values:

The next Block performs the MCMC interference and calculates error scores for the results.
Calculating the Energy-Score will take a lot of memory and computation power.
Its calculation can be disabled in the settings at the beginning.

In [5]:
# numerical evaluation:
for n_run in range(n_runs_MCMC):
    # generate random measurement from Prior:
    print(f'run: {n_run}')
    measurement_values = VM.generate_random_measurements(d_prior_dist)
    print(f'measurement values: ')
    for i, name in enumerate(measurement_names):
        print(f'{name}:   {measurement_values[i,0].numpy()}')

    '''
      creates a callable instance that returns the log-prob for a demand, given the prior and the measurements
      the class wrapper is used, as the functions in the tfp.mcmc can only take one input argument which is the sample
    '''
    log_prob_demand_inst = LogProbDemand(d_prior_dist, model, measurement_indices, measurement_values, measurement_stds)

    '''
        define kernel;
            method: Hamiltonian (Markov Chain) Monte Carlo,
            log_prob of posterior: log_prob_demand_inst
            automatic step-size adaption on the first 80% of the burnin steps
            target acceptance rate: 75% (default)
    '''
    def adaptive_hmc(num_burnin_steps):
        return tfp.mcmc.SimpleStepSizeAdaptation(
            tfp.mcmc.HamiltonianMonteCarlo(
            # tfp.experimental.mcmc.PreconditionedHamiltonianMonteCarlo(
                target_log_prob_fn=log_prob_demand_inst,
                num_leapfrog_steps=1,
                step_size=0.01),
            num_adaptation_steps=int(num_burnin_steps * 0.8))

    # this tf.function may trigger a warning for retracing. This retracing can not be avoided as the measurements may
    # change for each iteration of the loop  -  ignore the warning
    @tf.function
    def run_chain_hmc(initial_state, num_results, num_burnin_steps):
        samples, is_accepted = tfp.mcmc.sample_chain(
            num_results=num_results,
            num_burnin_steps=num_burnin_steps,
            current_state=initial_state,
            kernel=adaptive_hmc(num_burnin_steps),
            num_steps_between_results=1,
            trace_fn=lambda _, pkr: pkr)
        return samples, is_accepted

    # function to sample starting points, run the chains and return the demands + states
    def run_mcmc(num_burnin_steps, num_results):
        initial_state = d_prior_dist.sample(num_chains)
        initial_state = tf.Variable(initial_state)

        print('start calculating chains')
        chains, kernel_results = run_chain_hmc(initial_state, num_results, num_burnin_steps)
        # '''
        try:
            print('acceptance ratio: ', np.mean(kernel_results.is_accepted))
        except:
            print('acceptance ratio: ', np.mean(kernel_results.inner_results.is_accepted))

        dem_samples = tf.reshape(chains, shape=(-1, n_demands))
        MCMC_states = model(dem_samples)

        return dem_samples, MCMC_states, kernel_results


    print('++++++++++++++++++++++++++++++++++++++  Perform MCMC sampling   +++++++++++++++++++++++++++++++++++++++++++')

    demands_MCMC, states_MCMC, kernel_results = run_mcmc(int(n_burn_ins[0]), num_results)

    #%%
    print('++++++++++++++++++++++++++++++++++++++++  evaluate results   ++++++++++++++++++++++++++++++++++++++++++++++')

    # sampling importance resampling to gather ground truth results:
    weights = weight_samples(MC_states, measurement_indices, measurement_stds,
                             measurement_values, plot=False)
    n_resample = 10000
    weights = weights / np.sum(weights)             # normalise weights
    sample_ind = np.random.choice(len(weights), size=n_resample, p=weights)
    states_res  = tf.gather(MC_states, sample_ind)  # resampled states
    weights_res = tf.gather(weights, sample_ind)    # weights associated with each resampled state

    # remove constant dimensions before calculating the ED
    y_data =  tf.sparse.sparse_dense_matmul(states_res, SE.mask_matrix_full)
    y_model = tf.sparse.sparse_dense_matmul(states_MCMC, SE.mask_matrix_full)

    if calculate_ED:
        # ED over all dimensions
        ED[n_run] = ed(y_data=y_data, y_model=y_model)
        # ED for each type of state variable
        elementwise_ED[n_run, :] = segmented_ed(y_data=y_data, y_model=y_model, segments=SE.state_dimension_segments)
    # Quantile distances:
    for s in range(n_states):
        q5dist[n_run, s] = np.quantile(states_res[:, s], 0.05) - np.quantile(states_MCMC[:, s], 0.05)
        q5dist_rel[n_run, s] = q5dist[n_run, s] / np.quantile(states_res[:, s], 0.05)
        mean_dist[n_run, s] = (tf.reduce_mean(states_res[:, s]) - tf.reduce_mean(states_MCMC[:, s])).numpy()
    for s in range(n_states):
        q10dist[n_run, s] = np.quantile(states_res[:, s], 0.1) - np.quantile(states_MCMC[:, s], 0.1)
        q10dist_rel[n_run, s] = q10dist[n_run, s] / np.quantile(states_res[:, s], 0.1)

run: 0
measurement values: 
mf at edge_13:   7.622763770540886
T at DA1063:   65.01188468387012
++++++++++++++++++++++++++++++++++++++  Perform MCMC sampling   +++++++++++++++++++++++++++++++++++++++++++
start calculating chains
acceptance ratio:  0.73081
++++++++++++++++++++++++++++++++++++++++  evaluate results   ++++++++++++++++++++++++++++++++++++++++++++++
run: 1
measurement values: 
mf at edge_13:   7.653136073352052
T at DA1063:   65.66200551687463
++++++++++++++++++++++++++++++++++++++  Perform MCMC sampling   +++++++++++++++++++++++++++++++++++++++++++
start calculating chains
acceptance ratio:  0.76446
++++++++++++++++++++++++++++++++++++++++  evaluate results   ++++++++++++++++++++++++++++++++++++++++++++++


Report the results

create csv files of all important results and display the quantile distances and energy distances

In [6]:
pd.DataFrame(q5dist).to_csv(open(f'results/q5dist_{file_spec}.csv', 'w'), header=False, index=False)
pd.DataFrame(q10dist).to_csv(open(f'results/q10dist_{file_spec}.csv', 'w'), header=False, index=False)
pd.DataFrame(mean_dist).to_csv(open(f'results/meandist_{file_spec}.csv', 'w'), header=False, index=False)
pd.DataFrame(ED).to_csv(open(f'results/ed_{file_spec}.csv', 'w'), header=False, index=False)
pd.DataFrame(elementwise_ED).to_csv(open(f'results/ed_sep_{file_spec}.csv', 'w'), header=False, index=False)

print('\n \n ############################ results MCMC : ######################## \n \n')
print(f'ED:  {ED} \n')
print('q5-distances:')
beauty_report_qdists(q5dist, SE)
print('q10-distances:')
beauty_report_qdists(q10dist, SE)

print('\n \n ######################### relative results MCMC : ##################### \n \n')
print('q5-distances:')
beauty_report_qdists(q5dist_rel, SE, mode='%')
print('q10-distances:')
beauty_report_qdists(q10dist_rel, SE, mode='%')





 
 ############################ results MCMC : ######################## 
 

ED:  [[0.]
 [0.]] 

q5-distances:
MAE dist T 0.56 
Max dist T 3.00 
MAE dist mf 0.03 
Max dist mf 0.13 
MAE dist p 6.80
Max dist p 94.99
MAE dist T_end 0.44 
Max dist T_end 2.97 
q10-distances:
MAE dist T 0.52 
Max dist T 2.05 
MAE dist mf 0.04 
Max dist mf 0.14 
MAE dist p 3.81
Max dist p 42.05
MAE dist T_end 0.42 
Max dist T_end 2.06 

 
 ######################### relative results MCMC : ##################### 
 

q5-distances:
MAE dist T 0.01 %
Max dist T 0.05 %
MAE dist mf 0.10 %
Max dist mf 0.67 %
MAE dist p 0.00 %
Max dist p 0.01 %
MAE dist T_end 0.01 %
Max dist T_end 0.04 %
q10-distances:
MAE dist T 0.01 %
Max dist T 0.03 %
MAE dist mf 0.12 %
Max dist mf 0.70 %
MAE dist p 0.00 %
Max dist p 0.01 %
MAE dist T_end 0.01 %
Max dist T_end 0.03 %
